## Section 4.3

### Some examples for using Hyperopt

#### **1. A basic where we are optimizing, f(a,b) = a\*\*2 - b\*\*2**

In [3]:
from hyperopt import tpe, fmin, hp

def objective_func(args):
    a = args['a']
    b = args['b']    
    f = a**2 - b**2
    return f

range_a = hp.uniform('a', -2, 3)
range_b = hp.uniform('b', -1, 2)

space = {'a': range_a,
            'b': range_b}

best = fmin(objective_func, space, algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [00:00<00:00, 144.07trial/s, best loss: -3.9886784776719564]


#### **2. Algorithm selection and Hyperparameter Optimization using Hyperotp**

In [8]:
from hyperopt import tpe, fmin, hp
import math as m

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [9]:
space = hp.choice('classifier',[
        {'model': 'KNeighborsClassifier',
        'param': {'n_neighbors':
                        hp.choice('n_neighbors',range(3,11)),
        'algorithm':hp.choice('algorithm', ['ball_tree', 'kd_tree']),
        'leaf_size':hp.choice('leaf_size', range(1,50)),
        'metric':hp.choice('metric', ["euclidean", "manhattan",
                           "chebyshev", "minkowski"
                           ])}
        },
        {'model': 'SVC',
        'param':{'C':hp.loguniform('C', -2*m.log(10), 11*m.log(10)),
        'kernel':hp.choice('kernel',['rbf', 'poly', 'sigmoid']),
        'degree':hp.choice('degree', range(1,15)),
        'gamma':hp.loguniform('gamma', -9*m.log(10), 3*m.log(10))}
        }
        ])

In [10]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data,
                                                    digits.target,
                                                    test_size=0.3)

logs = {'args':list(),
        'train_score': list(),
        'val_score': list()}

def objective_func(args):
    clf_func = args["model"]
    params = args["param"]
    
    clf = eval(clf_func)(**params)
    clf.fit(X_train, y_train)
    
    val_score = clf.score(X_test, y_test)
    train_score = clf.score(X_train, y_train)
    
    logs['args'].append(args)
    logs['train_score'].append(train_score)
    logs['val_score'].append(val_score)
    
    return -val_score

best = fmin(objective_func, space, algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [00:31<00:00,  3.18trial/s, best loss: -0.9944444444444445]


In [11]:
best

{'C': 9674498124.113293,
 'classifier': 1,
 'degree': 6,
 'gamma': 0.0007494255146440473,
 'kernel': 0}

Here,
* 'classifier': 1, means SVC is the selected classifier
* 'degree': 6, is actually the 7th degree
* 'kernel': 0, means 'rbf'

Though we don't use degree with 'rbf' kernel so we can ignore degree

#### **3. Hyperopt with Keras, to optimize Neural Network Architecture**

In [12]:
from hyperopt import hp, tpe, fmin
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
import numpy as np

# load and preprocess the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
classes = 10
input_shape = 784
y_train = np_utils.to_categorical(y_train, classes)
y_test = np_utils.to_categorical(y_test, classes)

#logs
logs = {'model_summary':list(),
        'val_acc': list()}

def obj_func(args):
    
    model = Sequential()
    
    #defining first hidden layer
    model.add(Dense(units=args['units']['layer_units_1'], 
                    input_shape=(input_shape, ),
                    name='layer_units_1'))
    
    #defining number of remaining hidden layer
    number_of_layers = len(args['units'])
    for layer in range(2, number_of_layers):
        model.add(Dense(units=args['units'][f'layer_units_{layer}'], 
                        name=f'layer_units_{layer}'))
        model.add(Dropout(args['dropout'][f'dropout_p_{layer}'], 
                          name=f'dropout_p_{layer}'))
        model.add(Activation(activation=args['activation'][f'activation_{layer}'], 
                             name=f'activation_{layer}'))
        
    model.add(Dense(classes, name=f'layer_unit_{layer+1}'))
    model.add(Activation(activation='softmax', name=f'activation_{layer+1}'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer='adam')
    
    result = model.fit(x_train, y_train,
                      batch_size=2,
                      epochs=1,
                      verbose=3,
                      validation_split=0.2)
    
    validation_acc = np.amax(result.history['val_accuracy'])
    print(validation_acc)
    logs['model_summary'].append(model.summary())
    logs['val_acc'].append(validation_acc)
    
    return -validation_acc

def each_layer(number_of_layers):
    params = {'units': dict(),
              'dropout': dict(),
              'activation': dict()}
    number_of_nodes = [16,36,64,128,256,512]
    for layer in range(number_of_layers):
        params['units'][f'layer_units_{layer}'] = hp.choice(f'layer_{number_of_layers}_{layer}', 
                                                            number_of_nodes)
        params['dropout'][f'dropout_p_{layer}'] = hp.uniform(f'dropout_{number_of_layers}_{layer}', 
                                                             0, 
                                                             0.8)
        params['activation'][f'activation_{layer}'] = hp.choice(f'activation_{number_of_layers}_{layer}', 
                                                                ['relu', 'elu'])
    return params

number_of_layers = [3, 5, 7, 9]
space = hp.choice('layers', [each_layer(n) for n in number_of_layers])
best = fmin(obj_func, space, algo=tpe.suggest, max_evals=5)

0.132833331823349                                    
Model: "sequential"                                  
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_units_1 (Dense)        (None, 64)                50240     
_________________________________________________________________
layer_units_2 (Dense)        (None, 512)               33280     
_________________________________________________________________
dropout_p_2 (Dropout)        (None, 512)               0         
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
layer_units_3 (Dense)        (None, 128)               65664     
_________________________________________________________________
dropout_p_3 (Dropout)        (None, 128)               0         
__________________________________